In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import f1_score
from sklearn.svm import NuSVC
from sklearn.ensemble import StackingClassifier

# load dataset

In [2]:
df_independent = pd.read_csv('train_x_1.csv')

# preprocessing

droup unwanted feature

In [3]:
df_independent=df_independent.drop(["Unnamed: 0"], axis = 1)

In [4]:
df_independent

,item_id,shop_id,item_cnt_month
0,947,0,1
1,18462,0,1
2,15848,0,1
3,2416,0,2
4,1956,0,1
...,...,...,...
15085,5821,58,1
15086,9355,58,2
15087,17374,58,1
15088,2293,58,1


remove na from rows

In [5]:
df_independent.dropna(axis=0, #Determine if rows or columns which contain missing values are removed.
                 how='all', #Determine if row or column is removed from DataFrame
                 thresh=None, #Require that many non-NA values
                 subset=None, #
                 inplace=False)#If True, do operation inplace and return None.)

,item_id,shop_id,item_cnt_month
0,947,0,1
1,18462,0,1
2,15848,0,1
3,2416,0,2
4,1956,0,1
...,...,...,...
15085,5821,58,1
15086,9355,58,2
15087,17374,58,1
15088,2293,58,1


In [6]:
X_TRAIN_f1=df_independent.drop(["item_cnt_month"], axis = 1)
Y_TRAIN=df_independent.drop(["item_id","shop_id"], axis = 1)

In [7]:
from sklearn.model_selection import StratifiedKFold
skfold=StratifiedKFold(n_splits=5)


sagregate data of item sale for different different shops

# Prepareing model

data management

consider 30 sete oe daata as training set and  30 are testing parallelly
as train shop lavel 1 data and test it on shop lavel 2----set1
as train shop lavel 3 data and test it on shop lavel 4----set2
compare all sets score 


Model

prepareing weak models

In [8]:
#The Complement Naive Bayes classifier
""" CNB is an adaptation of the standard multinomial naive Bayes (MNB) algorithm that is 
particularly suited for imbalanced data sets. Specifically"""

mu2=ComplementNB(alpha=1.0, #Additive smoothing parameter
             fit_prior=True, # learn class prior probabilities
             class_prior=None, #Prior probabilities of the classes
             norm=True# second normalization of the weights is performed.
            )


#Naive Bayes classifier for multinomial models
""""The multinomial Naive Bayes classifier is suitable for classification with discrete features 
The multinomial distribution normally requires integer feature counts"""
mu=MultinomialNB(alpha=1.0, #Additive smoothing parameter
                 fit_prior=True, # learn class prior probabilities
                 class_prior=None #Prior probabilities of the classes
                )


cross-validation code

this is the subset accuracy which is a harsh metric since you require for each sample that each label set be correctly predicted.

In [9]:
df1=cross_val_score(mu, X_TRAIN_f1,Y_TRAIN,cv=skfold)

C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when

In [10]:
df2=cross_val_score(mu2, X_TRAIN_f1,Y_TRAIN)

C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when

avarage accuracy for the models

In [12]:
print("accuracy for CNB ",np.mean(df1), "and" ,"Naive Bayes",np.mean(df2))

accuracy for CNB  0.4133863485752154 and Naive Bayes 0.8845593108018555


StackingClassifier

In [13]:
estimators_1=[('accuracy',mu2),('Naive Bayes', mu)]

The optimal choice of the value  is highly data-dependent: in general a larger  suppresses the effects of noise, but makes the classification boundaries less distinct. Classifier implementing the k-nearest neighbors vote.

In [14]:
kn=KNeighborsClassifier(n_neighbors=5,#Number of neighbors to use by default for kneighbors queries
                     weights='distance',#weight points by the inverse of their distance.
                     algorithm='kd_tree',#Algorithm used to compute the nearest neighbors kd tree use for low dimension
                     leaf_size=30,
                     p=3,#Power parameter for the Minkowski metric. 
                     metric='minkowski' #the distance metric to use for the tree.
                    )

In [15]:
st=StackingClassifier(estimators=estimators_1, final_estimator=kn,cv=None, 
                   stack_method='predict')

In [16]:
df_s=cross_val_score(st, X_TRAIN_f1,Y_TRAIN)

C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which

In [17]:
print("accuracy for StackingClassifier ",np.mean(df_s))

accuracy for StackingClassifier  0.8322730284956925


# prepare and train models by verious training set 

In [18]:
df_tr1= pd.read_csv('train_x_1.csv')
df_tr2 = pd.read_csv('train_x_2.csv')
df_tr3 = pd.read_csv('train_x_3.csv')
df_tr4 = pd.read_csv('train_x_4.csv')

In [19]:
df_tr1=df_tr1.drop(["Unnamed: 0"], axis = 1)
df_tr2=df_tr2.drop(["Unnamed: 0"], axis = 1)
df_tr3=df_tr3.drop(["Unnamed: 0"], axis = 1)
df_tr4=df_tr4.drop(["Unnamed: 0"], axis = 1)

In [20]:
X_TR1=df_tr1.drop(["item_cnt_month"], axis = 1)
Y_TR1=df_tr1.drop(["item_id","shop_id"], axis = 1)

In [21]:
X_TR2=df_tr2.drop(["item_cnt_month"], axis = 1)
Y_TR2=df_tr2.drop(["item_id","shop_id"], axis = 1)

In [22]:
X_TR3=df_tr3.drop(["item_cnt_month"], axis = 1)
Y_TR3=df_tr3.drop(["item_id","shop_id"], axis = 1)

In [23]:
X_TR4=df_tr4.drop(["item_cnt_month"], axis = 1)
Y_TR4=df_tr4.drop(["item_id","shop_id"], axis = 1)

fit models to every set and compile it to make a finnal model

In [24]:
m_1=st.fit(X_TR1,Y_TR1) # 1st model
m_2=st.fit(X_TR2,Y_TR2) # 2nd model
m_3=st.fit(X_TR3,Y_TR3) # 3rd model
m_4=st.fit(X_TR4,Y_TR4) # 3rd model

C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expe

In [25]:
estimators=[('mode_1',m_1),('mode_2',m_2),('mode_3',m_3),('mode_4',m_4)]

In [26]:
final_model=StackingClassifier(estimators=estimators, final_estimator=kn,cv=None, 
                   stack_method='auto')

# testing of the model

In [27]:
df_test = pd.read_csv('test_x_1.csv')

In [28]:
df_test=df_test.drop(["Unnamed: 0"], axis = 1)

In [29]:
X_Ts=df_test.drop(["item_cnt_month"], axis = 1)
Y_Ts=df_test.drop(["item_id","shop_id"], axis = 1)

In [30]:
df_t=cross_val_score(final_model,X_Ts,Y_Ts)


C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which

C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_spli

C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_spli

C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_spli

C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_spli

C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_spli

C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_spli

C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_spli

C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_spli

In [31]:
m_4.named_estimators_

{'accuracy': ComplementNB(norm=True), 'Naive Bayes': MultinomialNB()}

In [32]:
m_4.final_estimator_

KNeighborsClassifier(algorithm='kd_tree', p=3, weights='distance')

In [33]:
print("accuracy for StackingClassifier ",np.mean(df_t))

accuracy for StackingClassifier  0.8851557322730285


model accuracy is overall performing so fit the model to relatively big chunk and preapre working model to export

In [34]:
df_TR= pd.read_csv('TRAIN_TO_FIT.csv')

In [35]:
df_TR=df_TR.drop(["Unnamed: 0"], axis = 1)

In [36]:
X_W=df_TR.drop(["item_cnt_month"], axis = 1)
Y_W=df_TR.drop(["item_id","shop_id"], axis = 1)

In [37]:
WORKING_MODEL=final_model.fit(X_W,Y_W)

C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which

C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_validation.py:999: RuntimeWarning: Number of classes in training fold (16) does not match total number of classes (17). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn('Number of classes in training fold ({}) does '
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python3

C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_validation.py:999: RuntimeWarning: Number of classes in training fold (15) does not match total number of classes (17). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn('Number of classes in training fold ({}) does '
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\USER\AppData\Local\Programs\Python\Python3

In [38]:
WORKING_MODEL.final_estimator_

KNeighborsClassifier(algorithm='kd_tree', p=3, weights='distance')

testing for 1 lavale shops

In [39]:
from joblib import dump
dump(WORKING_MODEL, 'WORKING_MODEL.joblib') 

['WORKING_MODEL.joblib']